# MNIST

### MNIST 데이터 받기

In [1]:
import os
import sys
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("data/", one_hot=True)
train_data = mnist.train.images
train_label = mnist.train.labels
test_data = mnist.test.images
test_label = mnist.test.labels

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


### MNIST 데이터 확인하기

In [2]:
# size of MNIST
print(train_data.shape)
print(train_label.shape)
print(test_data.shape)
print(test_label.shape)

(55000, 784)
(55000, 10)
(10000, 784)
(10000, 10)


### Data Pre-processing (데이터 전처리) : 다른 데이터로 하게 되면 해야됨

# Implementation (1)


## Loss function (손실 함수) : Cross Entropy

# <center> \\( L(y_i, f(x_i; W)) = -\frac{1}{n}\sum_{i=1}^{n}\sum_{k=1} y_{i,j} log(f(x_i)_k)\\)</center>


#### get_cross_entropy_loss 함수의 내용을 완성하세요.
#### (Hint : (1) tf.reduce_mean(), tf.reduce_sum(), tf.log() (2) Tensor dimension에 유의 (3) log 함수 사용 시 epsilon 사용하세요.)

In [3]:
def get_cross_entropy_loss(y_true, y_hat, epsilon=1e-8):
    """
    compute cross entropy

    Args:
        y_true: true label
        y_hat: predicted label
        epsilon: small value to prevent NaN in log

    Returns:
        cross entropy loss
    """
    with tf.name_scope('cross_entropy'):
#         loss = -tf.reduce_mean(tf.reduce_sum(y_true * tf.log(y_hat), axis=1)) #axis = 0은 세로, 1은 가로 행 하나를 집어서 열을 다 합침
        loss = -tf.reduce_mean(tf.reduce_sum(y_true * tf.log(y_hat), axis=1))
        return loss
    
# y_true = tf.placeholder(tf.float32, [None, 10]) # 샘플 갯수랑, 디멘젼
# y_hat = tf.placeholder(tf.float32, [None, 10]) # 샘플 갯수랑 디멘젼
# get_cross_entropy_loss(y_true, y_hat)

In [4]:

        

def get_accuracy(y_true, y_hat):
    """
    compute cross entropy

    Args:
        y_true: true label
        y_hat: predicted label
        epsilon: small value to prevent NaN in log

    Returns:
        cross entropy loss
    """
    with tf.name_scope('accuracy'):
        # Compare the highest indices between the predicted label and the true label
        correct_prediction = tf.equal(tf.argmax(y_hat, 1), tf.argmax(y_true, 1), name='correct_prediction')
        # Compute accuracy
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='accuracy')
    return accuracy

## Hyper-parameter (하이퍼 파라미터)

In [10]:
# Set hypyerparameters
learning_rate = 0.005
max_iter = 5000
batch_size = 100

# Implementation (2)
## Linear Classifier (선형 분류기)

## <center> \\( f(x) = W^Tx+b \\)</center>

### linear function을 완성하세요.
### (Hint : (1) weight, bias 선언 (2) tf.get_variable()의 initializer  (3) tf.matmul())

In [11]:
def fc(name, out_dim, inputs):
    """
    Args :
        Inputs : Input tensor
        out_dim : output dimension
        
    Returns:
        inputs * weight + bias
    """
    with tf.variable_scope(name):
        shp = inputs.get_shape().as_list()[1]
        initial = tf.truncated_normal([shp, out_dim], stddev=.01)
        weights = tf.get_variable('w', initializer=initial) #reuse하지 않는 이상 
        y_hat = tf.matmul(inputs, weights)
        return y_hat


In [12]:
# a = tf.placeholder(tf.float32, [None,784])
# a.get_shape().as_list()[1]

# Implementation (3)

## Model Setting

### 1. Training data 및 Test data의 각각의 image를 한 vector로 만들어서 train_data, test_data에 각각 저장하세요.
#### Hint) 데이터 차원.
### 2. Dataset로부터 받은 데이터(Image, label)를 담을 변수를 각각 x 및 y_true에 선언하세요.
#### Hint) tf.placeholder
### 3. Implementation (2)에서 구현한 linear classifier 함수값과 softmax 함수를 통한 prediction 값을 y_hat에 저장하세요.
#### Hint) tf.nn.softmax
### 4. 3으로부터 얻은 결과를 통해 Implementation (1)에서 구현한 loss function을 통해 얻은 loss를 cross_entropy에 저장하세요.

In [13]:
tf.reset_default_graph()
# Flatten data
###################################################################
#                    Implementation 3-1                           #
###################################################################
#train_data = None
#test_data = None

###################################################################
#                    Implementation 3-2                           #
###################################################################
x = tf.placeholder(tf.float32, [None, 784])
y_true = tf.placeholder(tf.float32, [None, 10])

###################################################################
#                    Implementation 3-3                          #
###################################################################
h1 = fc('layer1', 512, x)
y_logits = fc('layer2', 10, h1)
y_hat = tf.nn.softmax(y_logits)

###################################################################
#                    Implementation 3-4                           #
###################################################################
cross_entropy = get_cross_entropy_loss(y_true, y_hat)

# Calculate accuracy
accuracy = get_accuracy(y_true, y_hat)
# Make gradient descent op
train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

# Make op to initialize declared variable
init = tf.global_variables_initializer()

with tf.Session() as sess:
    # Initialize variables
    sess.run(init)
    
    # Training
    for step in range(max_iter):
        # Get batch data and label
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # train the network and calculate cross entropy
        _, loss = sess.run([train_step, cross_entropy], feed_dict={x: batch_x, y_true: batch_y})
        # calcualte accuracy
        acc = sess.run(accuracy, feed_dict={x: test_data, y_true: test_label})
        # print loss (cross entropy) and accuracy at every 10th step
        if (step + 1) % 10 == 0:
            print("{}th iteration, loss: {:.4f}, test accuracy: {:.4f}".format(step + 1, loss, acc))

10th iteration, loss: 0.6187, test accuracy: 0.8080
20th iteration, loss: 0.4590, test accuracy: 0.8560
30th iteration, loss: 0.7180, test accuracy: 0.8518
40th iteration, loss: 0.6949, test accuracy: 0.8606
50th iteration, loss: 0.4075, test accuracy: 0.8606
60th iteration, loss: 0.3394, test accuracy: 0.8695
70th iteration, loss: 0.3967, test accuracy: 0.8741
80th iteration, loss: 0.5362, test accuracy: 0.8810
90th iteration, loss: 0.3647, test accuracy: 0.8826
100th iteration, loss: 0.2775, test accuracy: 0.8824
110th iteration, loss: 0.6166, test accuracy: 0.8823
120th iteration, loss: 0.3619, test accuracy: 0.8852
130th iteration, loss: 0.4663, test accuracy: 0.8794
140th iteration, loss: 0.2061, test accuracy: 0.8914
150th iteration, loss: 0.4213, test accuracy: 0.8983
160th iteration, loss: 0.3478, test accuracy: 0.8738
170th iteration, loss: 0.4540, test accuracy: 0.8800
180th iteration, loss: 0.5929, test accuracy: 0.8926
190th iteration, loss: 0.3909, test accuracy: 0.8784
20

1560th iteration, loss: 0.3297, test accuracy: 0.9005
1570th iteration, loss: 0.1930, test accuracy: 0.9045
1580th iteration, loss: 0.2883, test accuracy: 0.8977
1590th iteration, loss: 0.3810, test accuracy: 0.9018
1600th iteration, loss: 0.2813, test accuracy: 0.8986
1610th iteration, loss: 0.2412, test accuracy: 0.9094
1620th iteration, loss: 0.3222, test accuracy: 0.9102
1630th iteration, loss: 0.5376, test accuracy: 0.9105
1640th iteration, loss: 0.4382, test accuracy: 0.8949
1650th iteration, loss: 0.3904, test accuracy: 0.8954
1660th iteration, loss: 0.2332, test accuracy: 0.9035
1670th iteration, loss: 0.7232, test accuracy: 0.9067
1680th iteration, loss: 0.4331, test accuracy: 0.8957
1690th iteration, loss: 0.2846, test accuracy: 0.9071
1700th iteration, loss: 0.2021, test accuracy: 0.8972
1710th iteration, loss: 0.4566, test accuracy: 0.9021
1720th iteration, loss: 0.2112, test accuracy: 0.9070
1730th iteration, loss: 0.4279, test accuracy: 0.9019
1740th iteration, loss: 0.37

3080th iteration, loss: 0.1771, test accuracy: 0.9117
3090th iteration, loss: 0.2927, test accuracy: 0.8990
3100th iteration, loss: 0.2298, test accuracy: 0.9106
3110th iteration, loss: 0.1768, test accuracy: 0.8945
3120th iteration, loss: 0.6634, test accuracy: 0.9072
3130th iteration, loss: 0.4138, test accuracy: 0.9002
3140th iteration, loss: 0.3034, test accuracy: 0.8973
3150th iteration, loss: 0.3121, test accuracy: 0.8992
3160th iteration, loss: 0.3090, test accuracy: 0.8968
3170th iteration, loss: 0.2368, test accuracy: 0.8945
3180th iteration, loss: 0.3661, test accuracy: 0.8958
3190th iteration, loss: 0.3602, test accuracy: 0.9014
3200th iteration, loss: 0.5323, test accuracy: 0.9023
3210th iteration, loss: 0.2000, test accuracy: 0.8961
3220th iteration, loss: 0.1973, test accuracy: 0.9047
3230th iteration, loss: 0.2946, test accuracy: 0.9022
3240th iteration, loss: 0.4499, test accuracy: 0.9007
3250th iteration, loss: 0.3114, test accuracy: 0.9132
3260th iteration, loss: 0.27

4680th iteration, loss: nan, test accuracy: 0.0980
4690th iteration, loss: nan, test accuracy: 0.0980
4700th iteration, loss: nan, test accuracy: 0.0980
4710th iteration, loss: nan, test accuracy: 0.0980
4720th iteration, loss: nan, test accuracy: 0.0980
4730th iteration, loss: nan, test accuracy: 0.0980
4740th iteration, loss: nan, test accuracy: 0.0980
4750th iteration, loss: nan, test accuracy: 0.0980
4760th iteration, loss: nan, test accuracy: 0.0980
4770th iteration, loss: nan, test accuracy: 0.0980
4780th iteration, loss: nan, test accuracy: 0.0980
4790th iteration, loss: nan, test accuracy: 0.0980
4800th iteration, loss: nan, test accuracy: 0.0980
4810th iteration, loss: nan, test accuracy: 0.0980
4820th iteration, loss: nan, test accuracy: 0.0980
4830th iteration, loss: nan, test accuracy: 0.0980
4840th iteration, loss: nan, test accuracy: 0.0980
4850th iteration, loss: nan, test accuracy: 0.0980
4860th iteration, loss: nan, test accuracy: 0.0980
4870th iteration, loss: nan, te